In [1]:
from scipy import stats
%matplotlib widget
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import matplotlib.cm as cm
from Funciones_auxiliares import Mat_to_dataframe,see_teams,get_cluster_indexs,plot_branch,plot_teams,find_mix
from scipy import cluster
from scipy.spatial.distance import mahalanobis

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import scipy as sp
import math


### Importamos la data

In [2]:
plt.close('all')
plt.clf()
plt.close()
data = pd.read_pickle('Datos/PCCIBNMZc')
data['Mean'] = data.Bulk.apply(lambda row: np.mean(row,axis = 0))
data['Mean'] = (data['Mean'] - data.Mean.apply(lambda row:np.mean(row)))/data.Mean.apply(lambda row:np.std(row))
del data['Bulk']
Initial = len(data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Separamos en Test y Train por sessiones

In [3]:
Sessions_list = data.PatientExperiment.unique()
select_random_sessions = np.random.choice(Sessions_list,(2,int(len(Sessions_list)/2)), replace = False)
Dfs = {}
for i,sessions in enumerate(select_random_sessions):
    Dfs[i] = data[data.PatientExperiment.isin(sessions)]
lenghts = np.array([len(i) for i in Dfs.values()])
    
for key,values in Dfs.items():
    print(len(values))

825
752


Calculamos la matriz de correlacion

## No tengo que entrenar nada, asumimos que el template matching para train es perfecto

Distancia Euclidea

In [4]:
def Train_test_noise(data):
    Sessions_list = data.PatientExperiment.unique()
    Train_list = np.random.choice(Sessions_list, size=int(len(Sessions_list)/2),replace = False)
    Train = data[data.PatientExperiment.isin(Train_list)]
    Test = data[~data.PatientExperiment.isin(Train_list)]
    Train = Train.reset_index().rename(columns = {'index':'old_index'})
    Test = Test.reset_index().rename(columns = {'index':'old_index'})
    
    df_train = Train.copy()
    df_test = Test.copy()
    
    Noise = df_train[df_train.bNoise == 1]
    Neuron = df_train[df_train.bNoise == 0]
    MU = df_train[df_train.bNoise == 2]

    df_train = pd.concat([Neuron,MU,Noise], ignore_index= True, sort = False)
    t = df_train[df_train.bNoise == 1].index[0]

    Noise = df_test[df_test.bNoise == 1]
    Neuron = df_test[df_test.bNoise == 0]
    MU = df_test[df_test.bNoise == 2]

    df_test = pd.concat([Neuron,MU,Noise], ignore_index= True, sort = False)
    t2 = df_test[df_test.bNoise == 1].index[0]


    return df_train,df_test,t2,t

df_train,df_test,t2,t = Train_test_noise(data)
print(len(df_train),len(df_test))
print(len(df_train.PatientExperiment.unique()),len(df_test.PatientExperiment.unique()))

424 1146
30 30




## Testeamos

In [5]:
from scipy.spatial.distance import squareform, pdist

def compute_dist(df_test,df_train):
    df = pd.concat([df_train,df_test],ignore_index=True,sort = True)
    r = len(df_train)
    s = len(df_test)
    Dist = pd.DataFrame(squareform(pdist(np.array(df.loc[:, 'Mean'].to_list()))),columns=df.index,index=df.index).values
    Dist = Dist[r:,:r]
    return Dist

Dist = compute_dist(df_test,df_train)


Vemos la matriz de distancia, la linea negra separa neuron-noise

In [6]:
fig,ax = plt.subplots()
fig.canvas.draw()

xlabels = ['neuron(MU)_train','noise_train']
ylabels = ['neuron(MU)_test','noise_test']

plt.xticks([t/2,(len(Dist[0])+t)/2],xlabels)
plt.yticks([t2/2,(len(Dist)+t2)/2],ylabels)

plt.title('Eucludean distance')

plt.hlines(t2,0,len(Dist[0])-1)
plt.vlines(t,0,len(Dist)-1)
plt.imshow(Dist)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

([<matplotlib.axis.XTick at 0x7ff135e1cf90>,
 <a list of 2 Text xticklabel objects>)

([<matplotlib.axis.YTick at 0x7ff135cb1150>,
 <a list of 2 Text yticklabel objects>)

Text(0.5, 1, 'Eucludean distance')

Ahora tenemos una matriz de distancias. Cada fila representa la distancia de un template test a cada uno de los templates de train. Computamos el minimo, y nos fijamos si es neuron o noise, entonces asignamos ese valor.

In [7]:
number_votes = 1
Minn = [np.argsort(i)[:number_votes][0] for i in Dist[:]]

In [8]:
def create_label(lista,t):
    label = np.zeros(len(lista),dtype = int)
    noises_index = np.where(lista > t)[0]
    for i in noises_index:label[i] = 1
    return label

label = create_label(Minn,t)

In [9]:
neurons = label[:t2]
noises = label[t2:]

total_neurons = len(neurons)
total_noises = len(noises)
neurons_as_noise = sum(neurons)
noises_found = sum(noises)

print('neurons as noise = ',neurons_as_noise/total_neurons*100,'({})'.format(neurons_as_noise))
print('Noise found = ', noises_found/total_noises*100, '({})'.format(noises_found))

neurons as noise =  25.268817204301076 (94)
Noise found =  82.04134366925064 (635)


In [10]:
neurons_as_noise_index = np.where(neurons == 1)[0]
noise_matched_index = [Minn[i] for i in neurons_as_noise_index]

plots_per_fig = 25
number_of_figs = len(neurons_as_noise_index)//plots_per_fig + 1
Figures = [plt.figure() for i in range(number_of_figs)]
axes = [fig.subplots(5,5).flatten() for fig in Figures]
for i,indexes in enumerate(zip(neurons_as_noise_index,noise_matched_index)):
    fig_to_plot = i//plots_per_fig
    ax_to_plot = i%plots_per_fig
    neuron_index,noise_index = indexes
    if noise_index < t:print(noise_index, 'index in df_train is not noise')
    axes[fig_to_plot][ax_to_plot].plot(df_test.iloc[neuron_index].Mean)
    axes[fig_to_plot][ax_to_plot].plot(df_train.iloc[noise_index].Mean)
    axes[fig_to_plot][ax_to_plot].set_title('{}, {}'.format(neuron_index,noise_index))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '0, 291')

...

Text(0.5, 1.0, '356, 214')

Text(0.5, 1.0, '364, 183')

Text(0.5, 1.0, '365, 187')

Text(0.5, 1.0, '366, 275')

Text(0.5, 1.0, '368, 291')

In [44]:
def do_one_time(number_votes = 3):
    df_train,df_test,t2,t = Train_test_noise(data)
    noises_train = len(df_train) - t #in train
    neurons_train = t #in train
    Dist = compute_dist(df_test,df_train)
    Minn = [np.argsort(i)[:number_votes] for i in Dist[:]]
    label = create_label(Minn,t)
    neurons = label[:t2]
    noises = label[t2:]
    
    neurons_as_noise_index = np.where(neurons == 1)[0]
    noise_matched_index = [Minn[i][0] for i in neurons_as_noise_index]
    

    
    with open('report euclidean','a') as f:
        for indexes in zip(neurons_as_noise_index,noise_matched_index):
            neuron_index,noise_index = indexes
            old_index_neuron = df_test.iloc[neuron_index].old_index
            old_index_noise = df_train.iloc[noise_index].old_index
            f.write(str(old_index_neuron) + ',' + str(old_index_noise) + '\n')
            
    
    total_neurons = len(neurons) #in test
    print('total neurons in test = ',total_neurons,'. This represents ', total_neurons/len(label)*100,' of the test size')
    total_noises = len(noises) # in test
    print('total noises in test = ',total_noises,'. This represents ', total_noises/len(label)*100,' of the test size')

        
    total_found = sum(label)
    print( 'The algorithm labeled ',total_found, 'clusters as noise')
    
    neurons_as_noise = sum(neurons)
    print('The algorithm incorrectely labeled',round(neurons_as_noise/total_neurons*100,2), 'of the neurons test (falses_positives)')
    noises_found = sum(noises)
    print('The algorithm correctely labeled',round(noises_found/total_noises*100,2), 'of the noises test (Hits)')
    
    with open('euclidean_log','a') as f:
            f.write(str(noises_train) + ',' + str(neurons_train) +','+ str(total_noises) + ',' + str(total_neurons) + ',' + str(neurons_as_noise) + ',' + str(noises_found) + ','+ str(total_noises - noises_found) + '\n')
            
    print('\n')
    print('\n')

    
    falses_positives = neurons_as_noise/total_neurons
    noise_found = noises_found/total_noises
    
    return falses_positives,noise_found

In [45]:
falses_positives_list = []
noise_found_list= []
for i in range(20):
    a,b =  do_one_time(number_votes=1)
    falses_positives_list.append(a)
    noise_found_list.append(b)

total neurons in test =  270 . This represents  33.047735618115055  of the test size
total noises in test =  547 . This represents  66.95226438188494  of the test size
The algorithm labeled  455 clusters as noise
The algorithm incorrectely labeled 10.74 of the neurons test (falses_positives)
The algorithm correctely labeled 77.88 of the noises test (Hits)




total neurons in test =  245 . This represents  35.714285714285715  of the test size
total noises in test =  441 . This represents  64.28571428571429  of the test size
The algorithm labeled  372 clusters as noise
The algorithm incorrectely labeled 10.61 of the neurons test (falses_positives)
The algorithm correctely labeled 78.46 of the noises test (Hits)




total neurons in test =  243 . This represents  34.51704545454545  of the test size
total noises in test =  461 . This represents  65.48295454545455  of the test size
The algorithm labeled  397 clusters as noise
The algorithm incorrectely labeled 18.93 of the neurons test (fa

In [46]:
print(np.mean(falses_positives_list),'pm',np.std(falses_positives_list))
print(np.mean(noise_found_list),'pm',np.std(noise_found_list))

0.16371771541543456 pm 0.04551990958207096
0.827866178358118 pm 0.032929686601613765


In [13]:
plt.close('all')
plt.clf()
plt.close()
data = pd.read_pickle('Datos/PCCIBNMZc')
data['Mean'] = data.Bulk.apply(lambda row: np.mean(row,axis = 0))
data['Mean'] = (data['Mean'] - data.Mean.apply(lambda row:np.mean(row)))/data.Mean.apply(lambda row:np.std(row))
Number_seesions = len(data.PatientExperiment.unique())
print('There are',Number_seesions,' sessions in the data')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

There are 60  sessions in the data


In [14]:
df = pd.read_csv('report euclidean',names=['neuron','noise'])
if not all(data.iloc[df.noise.values].bNoise == 1): print('There are neuron indexes in df.noise')
if not all(data.iloc[df.neuron.values].bNoise != 1): print('There are noises indexes in df.noise')

In [15]:
def plots_stuff(df,plot_cluster = False,save_name = '',hist_count = []):
    if not hasattr(df, 'Mean_nn'): 
        print('df has not Mean_nn (not normalized) attribute.. computing it from Bulk')
        df['Mean_nn'] = df.Bulk.apply(lambda row:np.mean(row,axis = 0))
    num_figs = len(df)//25+1
    Figures = [plt.figure(i,figsize = (20,10)) for i in range(num_figs)]
    axes = []
    ind_fig = 0
    print('Se realizaran '+str(num_figs) + 'figuras  con '+ str(plots_per_fig) + ' subfiguras cada una... \n')
    print('Figuras finalizadas:')
    for figure in Figures:
        axes.append(figure.subplots(5,5).flat)
        figure.tight_layout()
        figure.subplots_adjust(hspace=.3, wspace=.3)
    for i in range(len(df)):
        Fig_to_plot = i//plots_per_fig
        fig = axes[Fig_to_plot]
        ax_to_plot = i%plots_per_fig
        subplot = fig[ax_to_plot]
        if plot_cluster:
                for j in range(min(len(df.Bulk.iloc[i]),5000)):
                    subplot.plot(df.Bulk.iloc[i][j],'b',linewidth = 0.1)
                subplot.plot(df.Mean_nn.iloc[i],'k',linewidth = 1,label = '{}'.format(i))
        subplot.legend()
        if not hist_count == []:
            subplot.title.set_text(str(hist_count[i]))
        if Fig_to_plot == ind_fig  + 1:
            Figures[ind_fig].savefig(save_name + str(ind_fig))
            plt.close(str(ind_fig))
            ind_fig +=1
            print('\n Figura '+str(ind_fig) +' guardada')
        print(i)
    Figures[-1].savefig(save_name + str(ind_fig + 1))
    print('ultima figura guardada')
    plt.close(str(ind_fig+1))


In [16]:
df

,neuron,noise
0,56,1198
1,78,1137
2,215,265
3,240,290
4,242,292
...,...,...
1641,1156,304
1642,1160,468
1643,1220,556
1644,1236,1555


## Neuronas

Estudiemos primero las neuronas altamente correlacionadas para descartar que ruidos puestos como neuronas

In [52]:
hist_neuron = np.histogram(df.neuron.values,bins = np.arange(len(data)))
plt.figure(figsize = (30,10))
s = np.where(hist_neuron[0]>=1)
plt.bar(np.arange(len(s[0])),hist_neuron[0][s[0]])
plt.ylabel('counts')
plt.xlabel('index_neuron')
plt.title('How many times a neuron(MU) was near a noise?')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 3000x1000 with 0 Axes>

<BarContainer object of 188 artists>

Text(0, 0.5, 'counts')

Text(0.5, 0, 'index_neuron')

Text(0.5, 1.0, 'How many times a neuron(MU) was near a noise?')

In [18]:
neurons= np.where(hist_neuron[0]>0)[0]
neurons

array([   2,   11,   12,   34,   47,   48,   56,   72,   74,   77,   78,
         81,   89,   92,   98,  106,  116,  121,  123,  124,  126,  127,
        142,  157,  163,  209,  212,  215,  217,  223,  224,  229,  240,
        242,  245,  247,  261,  277,  278,  291,  296,  300,  323,  337,
        407,  426,  434,  438,  450,  462,  465,  473,  474,  475,  477,
        481,  488,  494,  511,  514,  517,  518,  519,  520,  521,  522,
        526,  530,  532,  535,  536,  541,  543,  549,  552,  553,  563,
        565,  568,  576,  578,  580,  596,  599,  600,  610,  649,  650,
        651,  658,  664,  675,  677,  691,  692,  693,  695,  698,  699,
        728,  732,  735,  740,  741,  742,  744,  748,  760,  762,  768,
        770,  773,  777,  779,  782,  803,  843,  852,  881,  882,  887,
        888,  935,  937,  939,  947,  951,  955,  965,  980,  991,  995,
       1003, 1004, 1005, 1014, 1019, 1026, 1044, 1046, 1062, 1063, 1068,
       1070, 1080, 1089, 1099, 1112, 1130, 1140, 11

Ploteamos las medias

In [19]:
plots_per_fig = 25
number_of_figs = len(neurons)//plots_per_fig + 1
Figures = [plt.figure() for i in range(number_of_figs)]
axes = [fig.subplots(5,5).flatten() for fig in Figures]
for i,index_neuron in enumerate(neurons):
    fig_to_plot = i//plots_per_fig
    ax_to_plot = i%plots_per_fig
    mean_neuron = data.iloc[index_neuron].Mean
    axes[fig_to_plot][ax_to_plot].plot(mean_neuron)
    axes[fig_to_plot][ax_to_plot].set_title('{}'.format(index_neuron))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Text(0.5, 1.0, '1426')

Text(0.5, 1.0, '1446')

Text(0.5, 1.0, '1483')

Text(0.5, 1.0, '1484')

Text(0.5, 1.0, '1514')

In [22]:
plt.close('all')
neurons_to_plot = data.iloc[neurons]
hist_neurons_correlated = hist_neuron[0][neurons]
plots_stuff(neurons_to_plot,True,'Figuras/Neurons_correlated_euclidean/',hist_count=hist_neurons_correlated)

df has not Mean_nn (not normalized) attribute.. computing it from Bulk


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

186


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


187


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.


ultima figura guardada


## Ruidos

Histograma indices vs counts. POdemos ver ciertos indices numerosamente cerca de neuronas.. Ruidos mal labeleados?

In [51]:
hist_noises = np.histogram(df.noise.values,bins = np.arange(len(data)))
plt.figure(figsize = (30,10))
s = np.where(hist_noises[0]>=1)
plt.bar(np.arange(len(s[0])),hist_noises[0][s[0]])
plt.ylabel('counts')
plt.xlabel('index_noise')
plt.title('How many times a noise were near to a neuron(MU)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 3000x1000 with 0 Axes>

<BarContainer object of 186 artists>

Text(0, 0.5, 'counts')

Text(0.5, 0, 'index_noise')

Text(0.5, 1.0, 'How many times a noise were near to a neuron(MU)')

In [24]:
noises = np.where(hist_noises[0]>0)[0]
noises

array([  24,   25,   29,   30,   31,   44,   46,   57,   68,   69,   71,
         73,   82,   87,  107,  110,  111,  115,  118,  133,  144,  151,
        158,  171,  178,  181,  197,  200,  201,  203,  207,  216,  221,
        225,  241,  243,  244,  246,  250,  253,  254,  255,  256,  257,
        259,  263,  264,  265,  266,  270,  271,  273,  280,  290,  292,
        293,  295,  297,  301,  304,  305,  306,  308,  311,  316,  362,
        372,  374,  376,  392,  400,  402,  403,  410,  422,  435,  468,
        470,  476,  483,  492,  498,  513,  548,  556,  557,  558,  569,
        571,  574,  588,  589,  592,  593,  603,  604,  605,  608,  612,
        614,  616,  621,  623,  635,  638,  640,  657,  668,  697,  702,
        710,  753,  763,  783,  784,  785,  786,  787,  795,  797,  812,
        815,  851,  900,  977,  988, 1009, 1010, 1030, 1031, 1032, 1034,
       1039, 1065, 1071, 1072, 1074, 1083, 1084, 1101, 1117, 1119, 1125,
       1135, 1137, 1163, 1165, 1169, 1171, 1179, 11

Ploteamos las medias

In [25]:
plots_per_fig = 25
number_of_figs = len(noises)//plots_per_fig + 1
Figures = [plt.figure() for i in range(number_of_figs)]
axes = [fig.subplots(5,5).flatten() for fig in Figures]
for i,index_noise in enumerate(noises):
    fig_to_plot = i//plots_per_fig
    ax_to_plot = i%plots_per_fig
    mean_noise = data.iloc[index_noise].Mean
    axes[fig_to_plot][ax_to_plot].plot(mean_noise)
    axes[fig_to_plot][ax_to_plot].set_title('{}'.format(index_noise))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Text(0.5, 1.0, '1533')

Text(0.5, 1.0, '1553')

Text(0.5, 1.0, '1554')

Text(0.5, 1.0, '1555')

Text(0.5, 1.0, '1568')

In [26]:
plt.close('all')
df_to_plot = data.iloc[noises]
hist_noises_correlated = hist_noises[0][noises]
plots_stuff(df_to_plot,True,'Figuras/Noises_correlated_euclidean/',hist_count=hist_noises_correlated)

df has not Mean_nn (not normalized) attribute.. computing it from Bulk


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

182


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


183


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


184
185
ultima figura guardada


Ploteamos el cluster entero.

### What if we remove those clusters wrong labeled?

In [ ]:
noise_indexes = [14,18,74,76,77,78,115,142,175,184]
neuron_indexes = [22,33,34,35,38,64,67,75,81]
mu_indexes = [25,38,64,67,127]

In [ ]:
data.loc[neurons[noise_indexes],'bNoise'] = 1
data.loc[noises[neuron_indexes],'bNoise'] = 0
data.loc[noises[mu_indexes],'bNoise'] = 2

In [ ]:
falses_positives_list = []
noise_found_list= []
for i in range(50):
    a,b =  do_one_time(number_votes=1,report_name='report Mahalanobis after cleaning')
    falses_positives_list.append(a)
    noise_found_list.append(b)

In [ ]:
print(np.mean(falses_positives_list),'pm',np.std(falses_positives_list))
print(np.mean(noise_found_list),'pm',np.std(noise_found_list))

RESULTADOS

## Estudiemos un poco mejor el histograma

In [28]:
plt.close('all')
df_to_plot = data.iloc[noises]


Hagamos un nuevo histograma, en el cual estudiamos el numero de noises que efectivamente se correlacionaron con neuronas distintas

In [29]:
hist_noises = {i:0 for i in df.noise.unique()}
for noise in hist_noises: 
    hist_noises[noise] = len(df[df.noise == noise].neuron.unique())

In [49]:
plt.figure(figsize = (30,10))
plt.bar(np.arange(len(hist_noises)),hist_noises.values())
plt.ylabel('Counts')
plt.xlabel('Index_noise')
plt.title('Number of neurons near each noise')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 3000x1000 with 0 Axes>

<BarContainer object of 186 artists>

Text(0, 0.5, 'Counts')

Text(0.5, 0, 'Index_noise')

Text(0.5, 1.0, 'Number of neurons near each noise')

In [31]:
index_noises = np.where(np.array(list(hist_noises.values())) > 4)[0]
u = np.array(list(hist_noises))[index_noises]
index_df_to_plot = [df_to_plot.index.get_loc(i) for i in u]
np.sort(index_df_to_plot)

array([ 17,  39,  41,  58,  84,  93,  99, 113, 114, 115, 131, 134, 144,
       150, 167, 169, 170, 173, 184])

In [32]:
hist_noises_index_to_plot = np.array(list(hist_noises))[index_noises]
neurons_near = [df[df.noise == i].neuron.unique() for i in hist_noises_index_to_plot]

In [33]:
len(neurons_near)

19

In [44]:
plt.close('all')
figs_to_plot = int((len(index_df_to_plot)/2)) + 1
Figures_list = [plt.subplots(2,2,figsize = (10,10)) for i in range(figs_to_plot)]
for i,index_noise in enumerate(index_df_to_plot):
    nfig = i//2
    row_axes = i%2
    print(row_axes)
    subplot_noise = Figures_list[nfig][1][row_axes][0]
    subplot_neuron = Figures_list[nfig][1][row_axes][1]
    subplot_noise.plot(df_to_plot.iloc[index_noise].Mean,label = str(index_noise))
    subplot_noise.legend()
    for index_neuron in neurons_near[i]:
        subplot_neuron.plot(data.loc[index_neuron].Mean,label = str(neurons_to_plot.index.get_loc(index_neuron)))
    subplot_neuron.legend()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

0


1


0


1


0


In [45]:
df_to_plot

,PatientExperiment,Channel,Cluster,ISI,Bulk,bNoise,Mean,ZeroCrossings
24,HEC016\016context1\016context1,4,1,"[0, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, ...","[[-11.04594160592383, -12.25477139504218, -6.9...",1,"[0.3621592021384227, 0.52060388717631, 0.59160...",8
25,HEC016\016context1\016context1,4,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[7.772686848887978, 9.245231460208272, 10.297...",1,"[0.29254279272264483, 0.3628647890380416, 0.41...",7
29,HEC016\016context1\016context1,6,3,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[[9.673234198605336, 5.602644485619004, -1.786...",1,"[0.5283182144154817, 0.5433766397675794, 0.534...",2
30,HEC016\016context1\016context1,7,1,"[0, 0, 1, 2, 2, 3, 3, 5, 5, 4, 10, 3, 7, 6, 2,...","[[0.7283178801562687, -2.6019972212879776, -1....",1,"[0.4835945937521131, 0.5177982302430609, 0.528...",2
31,HEC016\016context1\016context1,8,1,"[0, 1, 3, 3, 0, 1, 9, 4, 0, 1, 1, 2, 1, 0, 0, ...","[[-5.014857376594767, -2.6034566651464575, 1.3...",1,"[0.4800081379931437, 0.5875605454364761, 0.593...",6
...,...,...,...,...,...,...,...,...
1533,HEC030\HEC030_story1,73,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[67.22831729493394, 58.59860162468246, 55.998...",1,"[1.0107445443258793, 1.0013258500328381, 0.997...",4
1553,HEC030\HEC030_story4,95,3,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...","[[30.56617779129116, 26.846576314233253, 15.50...",1,"[0.6329774241252774, 0.8106414791428904, 0.966...",3
1554,HEC030\HEC030_story4,98,2,"[0, 5, 6, 11, 2, 8, 10, 10, 15, 6, 19, 15, 15,...","[[9.248022125587147, 12.139254211247511, 13.79...",1,"[0.5850305135481051, 0.5912792052355617, 0.594...",2
1555,HEC030\HEC030_story5,100,3,"[0, 11, 6, 4, 4, 2, 6, 1, 1, 0, 2, 0, 4, 3, 1,...","[[-4.593516102357167, -3.010456316848897, -2.3...",1,"[-0.8672886383504445, -0.8424967468495845, -0....",3


In [ ]:
plt.close('all')
df_to_plot = data.iloc[noises]

In [33]:
falses_positives_list

[0.1092896174863388,
 0.10091743119266056,
 0.13380281690140844,
 0.15172413793103448,
 0.12886597938144329,
 0.12142857142857143,
 0.20930232558139536,
 0.19433198380566802,
 0.13815789473684212,
 0.20481927710843373,
 0.2638888888888889,
 0.11262798634812286,
 0.18518518518518517,
 0.15056818181818182,
 0.15358361774744028,
 0.1927710843373494,
 0.13829787234042554,
 0.15693430656934307,
 0.1937984496124031,
 0.17434210526315788]